In [1]:
import twitter
import json
import sys
import time
from urllib.error import URLError
from http.client import BadStatusLine
import pandas as pd
import csv

'''
    Self-created login function for access the data throught Twitter API
'''


def oauth_login():
    CONSUMER_KEY = 'WBHkhQ3mw9ogV0Rd2XXFAq8rd'
    CONSUMER_SECRET = '0O5PuMVxoumlUkYYGaV0IJEuBk7CN4djztz9RpTzFUJYUDhsVe'
    OAUTH_TOKEN = '1228341286886924289-rpd0vcn8BJLPVlOB5kKfzLuI2mCGCP'
    OAUTH_TOKEN_SECRET = 'SwEpzMxdROIBWl3HC7nCsU8VfhPLcNb5hcy89SfC3qrXP'
    auth = twitter.oauth.OAuth(OAUTH_TOKEN,OAUTH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
    twitter_api = twitter.Twitter(auth = auth)
    return twitter_api


'''
    Directly copied from the Cookbook provided
'''


def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw):

    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):

        if wait_period > 3600:  # Seconds
            print('Too many retries. Quitting.', file=sys.stderr)
            raise e

        # See https://developer.twitter.com/en/docs/basics/response-codes
        # for common codes

        if e.e.code == 401 or e.e.code == 403:
            print('Encountered 401 Error (Not Authorized)', file=sys.stderr)
            return None
        elif e.e.code == 404:
            print('Encountered 404 Error (Not Found)', file=sys.stderr)
            return None
        elif e.e.code == 429:
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print("Retrying in 15 minutes...ZzZ...", file=sys.stderr)
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print('...ZzZ...Awake now and trying again.', file=sys.stderr)
                return 2
            else:
                raise e  # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print('Encountered {0} Error. Retrying in {1} seconds'                  .format(
                e.e.code, wait_period), file=sys.stderr)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function

    wait_period = 2
    error_count = 0

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError as e:
            error_count = 0
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print("BadStatusLine encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise


'''
    Directly copied from the Cookbook provided by professor
'''


def get_user_profile(twitter_api, screen_names=None, user_ids=None):

    # Must have either screen_name or user_id (logical xor)
    assert (screen_names != None) != (
        user_ids != None),     "Must have screen_names or user_ids, but not both"

    items_to_info = {}

    items = screen_names or user_ids

    while len(items) > 0:

        # Process 100 items at a time per the API specifications for /users/lookup.
        # See http://bit.ly/2Gcjfzr for details.

        items_str = ','.join([str(item) for item in items[:100]])
        items = items[100:]

        if screen_names:
            response = make_twitter_request(twitter_api.users.lookup,
                                            screen_name=items_str)
        else:  # user_ids
            response = make_twitter_request(twitter_api.users.lookup,
                                            user_id=items_str)

        if response == None:
            continue

        for user_info in response:
            if screen_names:
                items_to_info[user_info['screen_name']] = user_info
            else:  # user_ids
                items_to_info[user_info['id']] = user_info

    return items_to_info


In [6]:
if __name__ == "__main__":
    twitter_api = oauth_login()
    twitter_ids = []
    with open('SamplingTweetID.txt', 'r') as file:
        data = file.read().replace('\n', ' ')
        twitter_ids = data.split(" ")
    user_ids = set()
    f = open("NewTweetInfo.csv", "a")
    fieldnames = ['Tweet_time', 'Tweet_id','Tweet_content','UserId','UserLocation','FollowerCount','RetweetTime']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for tid in twitter_ids[1:]:
        tweet = make_twitter_request(twitter_api.statuses.show, id=tid)
        if tweet != None:
            cur = str(tweet["created_at"]) + "\t" +"'"+ str(tweet["id"])+ "\t" + str(tweet["text"])+ "\t" +"'"+str(tweet["user"]["id"])+ "\t" + \
                str(tweet["user"]["location"])+ "\t" + str(tweet["user"]["followers_count"])+ "\t" + str(tweet["retweet_count"])
            cur = cur.split("\t")
            writer = csv.writer(f)
            writer.writerows([cur])
    # Self-crated file to store the tweet_time, tweet_id, tweet_message, userId, userLocation,userFollower, retweetCount
    f.close()

Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 4

Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not 

Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 

Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 E

Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized

...

Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 

Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found

Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encount

Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encount

Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 404 Error (Not Found)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encountered 401 Error (Not Authorized)
Encount